In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

def classify_event(image_path):
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 3)

    model.load_state_dict(torch.load('/content/drive/MyDrive/dataset/pred_img_type.path'))
    model.eval()

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(image)
        probabilities = nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

    event_list = ["BARGES", "BOATS", "DINGHY"]
    event = event_list[predicted.item()]

    # Display the probabilities for each class
    for i, prob in enumerate(probabilities[0]):
        print(f"{event_list[i]}: {prob.item() * 100:.2f}%")

    return event


In [ ]:
image_path = '/content/unknown.jpg'  # Replace with the path to your image
event = classify_event(image_path)
print(f"Predicted Event: {event}")


BARGES: 99.99%
BOATS: 0.00%
DINGHY: 0.01%
Predicted Event: BARGES
